## Подготовка

### Библиотеки

In [ ]:
import telebot
from telebot import types
import threading
import time
import os
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
import pandas as pd
import requests
import joblib
from talib import abstract
import pandas as pd
import warnings
import numpy as np
import oandapyV20
from datetime import datetime as Ddtime
from datetime import timezone
from datetime import timedelta
import tracemalloc
import oandapyV20.endpoints.instruments as instruments
from concurrent.futures import ThreadPoolExecutor, as_completed
from ratelimit import limits, sleep_and_retry
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Токены/ключи/id

In [ ]:
### Telegram token
TOKEN = 'PASTE_YOUR_TOKEN_HERE'
bot = telebot.TeleBot(TOKEN)

### Oanda token
access_token = 'PASTE_YOUR_TOKEN_HERE'
client = oandapyV20.API(access_token=access_token, environment="practice")

### Установка прогнозной модели
model = joblib.load("v2INFERENCE_CatBoost(1, 1.4)#lr0.03#d6#leg0.6#it600.joblib")

### Подключение к Google Sheets API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

json_google_sheets = 'PASTE_YOUR_GOOGLE_SHEETS_JSON_FILE_NAME_HERE'
key_google_sheets = 'PASTE_YOUR_GOOGLE_SHEETS_KEY_HERE'

credentials = ServiceAccountCredentials.from_json_keyfile_name(json_google_sheets, scope)
client_excel = gspread.authorize(credentials)
spreadsheet = client_excel.open_by_key(key_google_sheets)


### Названия валютных пар для получения сигналов + те, по которым есть scaler'ы
pari = """eurusd
audusd
audjpy
cadjpy
chfjpy
euraud
eurcad
eurgbp
gbpusd
usdcad
usdchf
usdjpy
cadchf"""

Igor_appended = ['AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDUSD', 'CADCHF', 'CADJPY',
                 'CHFJPY', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURUSD',
                 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPJPY', 'GBPUSD', 'USDCAD',
                 'USDCHF', 'USDJPY']

### Утилиты

In [ ]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


termintate = {}
chat_states = {}
registered_users = {}
has_started = ""
names = ["Частота напоминания о работе (сек.)", "Валютная пара"]
starting_point = ""
dict_all_curr = {}
glob_dict = {}
predicting = 1
accounted = 0
global_poll_curr = 0
once_started = 0

pari = pari + "\n" + "\n".join(Igor_appended)
pari = pari + "\n" + "\n".join([x[3:] + x[:3] for x in pari.split("\n")])
pari = [(x[:3].upper(), x[3:].upper()) for x in pari.split("\n")]
pari = list(set(pari))
txt_pari = "\n".join([" -> ".join(x) for x in pari])

scaler = {x: None for x in pari}
for x in pari:
    lab = x[0] + x[1]
    if os.path.exists("v2scalers/" + "scaler_"+ lab + ".bin"):
        scaler[x] = joblib.load(os.path.join("v2scalers/" + "scaler_"+ lab + ".bin"))
    else:
        scaler[x] = None

        
pari = [(x[0][0], x[0][1]) for i, x in enumerate(list(scaler.items())) if x[1] != None]
txt_pari = "\n".join([" -> ".join(x) for x in pari])
currency_pairs = txt_pari.replace(" -> ", "_").split("\n")

dict_time_frame = {}
storaging = []
time_delta = 6
global_waiting = 3



features = ['log_(close_t/close_t-3)',
 'CCI_14_+_WILLR_14',
 '(close_t + close_t-1) > (close_t-2 + close_t-3)',
 'log_(close_t/close_t-2)',
 'CCI_14_+_MAX/MIN',
 'CCI_14',
 'CCI_14_+_MINUS_DM_12',
 'CCI_14_-_MINUS_DM_12',
 'MINUS_DM_12_-_CCI_14',
 'CCI_14_x_MAX/MIN',
 'log_volume',
 'CCI_14_x_WILLR_14',
 '(close_t + close_t-1 + close_t-2) > (close_t-3 + close_t-4+ close_t-5)',
 'log_delta_p2_MACD/close',
 'close_t/close_t-3',
 'DX_12_-_CCI_14',
 'MAX/MIN_-_PLUS_DM_12',
 'CCI_14_-_DX_12',
 'MACD/close',
 'delta_p2_close_x_WILLR_14',
 'log_delta_p1_MACD/close',
 'log_close',
 'log_(close_t/close_t-1)',
 'avg_сlose_3 - avg_close_3_shift_3',
 'MAX/MIN_-_CCI_14',
 'delta_p1_volume_x_delta2_p1_volume',
 'PLUS_DM_12_x_RSI_14',
 'MINUS_DM_12_+_MAX/MIN',
 'log_delta2_p2_volume',
 'delta_p2_close_x_RSI_14',
 'delta_p1_close_x_delta_p2_volume',
 'CCI_14_+_MFI_30',
 'delta_p2_close_x_MFI_30',
 'CCI_14_-_MAX/MIN',
 'MFI_30_-_CCI_14',
 'PLUS_DM_12_-_MAX/MIN',
 'CCI_14_+_RSI_14',
 'MINUS_DM_12_x_WILLR_14',
 'log_delta2_p1_MACD/close',
 'delta_p1_volume_x_delta2_p2_volume',
 'log_delta_p2_close_x_delta2_p2_volume',
 'WILLR_14_-_CCI_14',
 'PLUS_DM_12',
 'close_t/close_t-1',
 'log_MACD/close',
 'delta_p2_close_x_MAX/MIN',
 'CCI_14_x_DX_12',
 'delta_p1_volume_x_delta_p2_volume',
 'PLUS_DM_12_-_RSI_14',
 'PLUS_DM_12_x_MAX/MIN',
 'delta_p2_close_x_MINUS_DM_12',
 'delta2_p2_close_x_DX_12',
 'CCI_14_+_PLUS_DM_12']

## Бот

### Часть с функциональным меню

In [ ]:
def convert(response):
    rsponse_ = response.text[1:-1]
    dc = rsponse_[rsponse_.find("{") + 1 : rsponse_.find("}")].replace(",", "\t")
    rs_mod = rsponse_[:rsponse_.find("{")] + dc + rsponse_[rsponse_.find("}") + 1:]
    mas = [x for x in rs_mod.split(",")]
    mas = [[x[:x.find(":")], x[x.find(":") + 1 :]] for x in mas]
    for i in range(len(mas)):
        mas[i][0] = mas[i][0].replace('"', '')
        el = mas[i][1].replace('"', '')
        if mas[i][0] == "ms":
            el = float(el)
        if mas[i][0] == "updated":
            el = pd.to_datetime(el)
        if mas[i][0] == "results":
            el = el.split("\t")
            mm = [tuple(k.split(":")) for k in el]
            dic = dict([(j[0], float(j[1])) for j in mm])
            el = dic
        if mas[i][0] == "result":
            el = [el]
            mm = [tuple(k.split(":")) for k in el]
            dic = dict([(j[0], float(j[1])) for j in mm])
            el = dic
        mas[i][1] = el
    output_text = dict([tuple(x) for x in mas])
    return output_text


def convert_to_beatiful(response):
    to_out = convert(response)
    basic = to_out["base"]
    dic = to_out["results"]
    date_time = to_out["updated"]
    txt_cur = "\n".join([f"{basic} -> {x[0]}: {x[1]}" for x in list(dic.items())])
    txt_date = date_time.strftime(format = "Дата: %d.%m.%Y,\nВремя: %H:%M:%S\n")
    string_txt = txt_date +"-"*18 + "\n" + txt_cur + "\n"
    return string_txt


def create_command_keyboard():
    markup = ReplyKeyboardMarkup(row_width=2, resize_keyboard = True)
    markup.add(KeyboardButton('/reg'))
    markup.add(KeyboardButton('/info'), KeyboardButton('/start'))
    markup.add(KeyboardButton('/stop'), KeyboardButton('/stopping'))
    return markup


@bot.message_handler(commands=['stopping'], func = lambda x: not termintate.get(x.chat.id, False))
def handle_stopping(message):
    global termintate, registered_users, necessary, has_started
    chat_id = message.chat.id

    to_delete = [x for x in os.listdir() if x[x.rfind(".") - 3:x.rfind(".")] == x[x.rfind(".") - 3:x.rfind(".")][0]*3]

    for file_name in to_delete:
        if os.path.exists(file_name):
            os.remove(file_name)

    termintate[chat_id] = True
    bot.reply_to(message, "Деятельность бота остановлена полностью.\nНа головном компьютере следует запустить код заново.",
                    reply_markup=create_command_keyboard())


@bot.message_handler(commands=['reg'], func = lambda x: not termintate.get(x.chat.id, False))
def handle_registration(message):
    chat_id = message.chat.id

    if chat_id not in registered_users:
        bot.send_message(chat_id, "Укажите частоту поступления сообщения\nо работе бота (целое число в секундах):")
        chat_states[chat_id] = "waiting_for_timeout"
        registered_users[chat_id] = {"timeout": None, "curr": []}
        return

    bot.reply_to(message, "Вы уже зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                 reply_markup=create_command_keyboard())


@bot.message_handler(func=lambda message: chat_states.get(message.chat.id) == "waiting_for_timeout" and not termintate.get(message.chat.id, False))
def handle_timeout(message):
    chat_id = message.chat.id
    global txt_pari, global_poll_curr
    try:
        timeout = int(message.text)
        registered_users[chat_id]["timeout"] = timeout
        tt = txt_pari.split("\n")
        chat_states[chat_id] = "waiting_for_curr"
        kb = types.InlineKeyboardMarkup(row_width = 4)
        for x in tt:
            if len(registered_users[chat_id]["curr"]) == len(tt):

                bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                         reply_markup=create_command_keyboard())
                chat_states.pop(chat_id)
                print(registered_users)
                break
            elif tuple(x.split(" -> ")) not in registered_users[chat_id]["curr"]:

                kb.add(types.InlineKeyboardButton(text = x, callback_data = x))
        kb.add(types.InlineKeyboardButton(text = "Выбрать всё", callback_data = "Выбрать всё"))
        kb.add(types.InlineKeyboardButton(text = "Выход", callback_data = "Выход"))
        global_poll_curr = bot.send_message(chat_id, "Выберете валютную пару из списка:", reply_markup = kb)
    except ValueError:
        bot.send_message(chat_id, "Пожалуйста, введите целое число в секундах.")
    

@bot.message_handler(func=lambda message: chat_states.get(message.chat.id) == "waiting_for_curr" and not termintate.get(message.chat.id, False))
def handle_curr(message):
    chat_id = message.chat.id
    global txt_pari, global_poll_curr
    bot.delete_message(chat_id, global_poll_curr.id)
    bot.send_message(chat_id, "Пожалуйста, выберете курс из данного списка.\nНе стоит писать что-либо.")
    tt = txt_pari.split("\n")

    kb = types.InlineKeyboardMarkup(row_width = 4)
    for x in tt:
        if len(registered_users[chat_id]["curr"]) == len(tt):
            bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                        reply_markup=create_command_keyboard())
            chat_states.pop(chat_id)
            print(registered_users)
            break
        elif tuple(x.split(" -> ")) not in registered_users[chat_id]["curr"]:

            kb.add(types.InlineKeyboardButton(text = x, callback_data = x))
    kb.add(types.InlineKeyboardButton(text = "Выход", callback_data = "Выход"))
    global_poll_curr = bot.send_message(chat_id, "Выберете валютную пару из списка:", reply_markup = kb)


@bot.callback_query_handler(func=lambda callback: callback.data != "Выход" and callback.data != "Выбрать всё" and not termintate.get(callback.message.chat.id, False))
def check_callback_data_in(callback):
    global txt_pari, global_poll_curr
    tt = txt_pari.split("\n")
    chat_id = callback.message.chat.id

    registered_users[chat_id]["curr"].append(tuple(callback.data.split(" -> ")))
    bot.edit_message_text(chat_id = chat_id, message_id = callback.message.id, text = f"Выбранная валютная пара: {callback.data}")

    kb = types.InlineKeyboardMarkup(row_width = 4)
    for x in tt:
        if len(registered_users[chat_id]["curr"]) == len(tt):
            bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                     reply_markup=create_command_keyboard())
            chat_states.pop(chat_id)
            print(registered_users)
            break
        elif tuple(x.split(" -> ")) not in registered_users[chat_id]["curr"]:
            kb.add(types.InlineKeyboardButton(text = x, callback_data = x))
    if len(registered_users[chat_id]["curr"]) != len(tt):
        kb.add(types.InlineKeyboardButton(text = "Выход", callback_data = "Выход"))
        global_poll_curr = bot.send_message(chat_id, "Выберете валютную пару из списка:", reply_markup = kb)


@bot.callback_query_handler(func=lambda callback: callback.data == "Выбрать всё" and not termintate.get(callback.message.chat.id, False))
def check_callback_data_in(callback):
    global txt_pari, global_poll_curr, pari
    tt = txt_pari.split("\n")
    chat_id = callback.message.chat.id

    registered_users[chat_id]["curr"] = pari
    bot.edit_message_text(chat_id = chat_id, message_id = callback.message.id, text = f"Выбранная валютная пара: {callback.data}")

    bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                reply_markup=create_command_keyboard())
    chat_states.pop(chat_id)
    print(registered_users)


@bot.callback_query_handler(func=lambda callback: callback.data == "Выход" and not termintate.get(callback.message.chat.id, False))
def check_callback_data_out(callback):
    global txt_pari
    chat_id = callback.message.chat.id
    if len(registered_users[chat_id]["curr"]) == 0:
        tt = txt_pari.split("\n")
        bot.edit_message_text(chat_id = chat_id, message_id = callback.message.id, text = "Пожалуйста, выберете хотя бы одну валютную пару.")
        kb = types.InlineKeyboardMarkup(row_width = 4)
        for x in tt:
            if len(registered_users[chat_id]["curr"]) == len(tt):
                bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                         reply_markup=create_command_keyboard())
                chat_states.pop(chat_id)
                break
            elif tuple(x.split(" -> ")) not in registered_users[chat_id]["curr"]:
                kb.add(types.InlineKeyboardButton(text = x, callback_data = x))
        kb.add(types.InlineKeyboardButton(text = "Выбрать всё", callback_data = "Выбрать всё"))
        kb.add(types.InlineKeyboardButton(text = "Выход", callback_data = "Выход"))
        bot.send_message(chat_id, "Выберете валютную пару из списка:", reply_markup = kb)
        return
    chat_states.pop(chat_id)
    print(registered_users)
    bot.edit_message_text(chat_id = chat_id, message_id = callback.message.id, text = f"Выбранная валютная пара: {callback.data}")
    bot.send_message(chat_id, "Вы зарегистрированы.\nЗарегистрироваться заново: /re_reg\nПолучить информацию о регистрации: /info",
                         reply_markup=create_command_keyboard())


@bot.message_handler(func=lambda message: message.chat.id not in registered_users and not termintate.get(message.chat.id, False))
def handle_unregistered(message):
    bot.reply_to(message, "Пожалуйста, зарегистрируйтесь с помощью команды /reg.")


@bot.message_handler(commands=['re_reg'], func = lambda message: message.chat.id in registered_users and not termintate.get(message.chat.id, False))
def handle_registration(message):
    
    global chat_states, registered_users, has_started

    chat_id = message.chat.id

    if chat_states.get(message.chat.id, False) == True:
        has_started = ""

        for para in registered_users[chat_id]["curr"]:
            file_name = f"{chat_id}_{para[0] + para[1]}_ccc.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{chat_id}_{para[0] + para[1]}_bbb.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{chat_id}_aaa.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{chat_id}_{para[0] + para[1]}_ddd.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            
        chat_states[chat_id] = False
        bot.reply_to(message, "Программа остановлена.\nДля начала работы введите /start,\nдля полного прекращения - /stopping.",
                    reply_markup=create_command_keyboard())

    registered_users.pop(chat_id)
    bot.send_message(chat_id, "Данные о регистрации аннулированы.\nЗарегестрируйтесь повторно с помощью /reg")


@bot.message_handler(commands=['info'], func = lambda message: message.chat.id in registered_users and not termintate.get(message.chat.id, False))
def handle_registration(message):
    global names, registered_users
    chat_id = message.chat.id
    out = [f"{names[i]}: {x[1]}" for i, x in enumerate(list(registered_users[chat_id].items()))]
    bot.send_message(chat_id, "Данные о регистрации:\n" + "\n".join(out))


### Часть с работой по парсингу данных, их обработке и выводу пользователю

In [ ]:
@bot.message_handler(commands=['start'], func = lambda x: not termintate.get(x.chat.id, False))
def handle_start(message):
    global chat_states, termintate, registered_users, necessary, has_started, predicting, accounted, dict_time_frame, storaging
    
    chat_id = message.chat.id
    if has_started == "":

        has_started = chat_id
        if chat_id not in chat_states:
            chat_states[chat_id] = False

        if not chat_states[chat_id]:
            bot.reply_to(message, "Понял команду /start. Готов начать работу.")
            chat_states[chat_id] = True
            
            dict_time_frame = {x: {"currency": x[0] + "_" + x[1],
                                   "tf_pred": -1,
                                   "predicted": -1,
                                   "tf_buy": -1,
                                   "buy_price": -1,
                                   "tf_sell": -1,
                                   "sell_price": -1,
                                   "winning": -1} for x in registered_users[chat_id]["curr"]}

            """
            for para in registered_users[chat_id]["curr"]:
                
                file_name = f'probab/{para[0] + para[1]}_probab.txt'
                file_read = open(file_name, "w+", encoding = "utf-8")
                file_read.close()
            """

            threading.Thread(target = run_bot, args=(message,)).start()
            threading.Thread(target = parse_oanda, args=(message,)).start()
            threading.Thread(target = osnova_oanda, args=(message,)).start()
            threading.Thread(target = storaging_oanda, args=(message,)).start()
    elif has_started == chat_id:
        bot.reply_to(message, "Бот уже работает!")
    else:
        bot.reply_to(message, "Команда воспроизводится на другом устройстве.\nПодождите, пока исполнение на нем закончится.")



def run_bot(message):
    global chat_states, termintate, registered_users, necessary

    chat_id = message.chat.id
    while chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users:
        bot.send_message(chat_id, "Бот работает. Для завершения работы введите /stop.")
        time.sleep(registered_users[chat_id]['timeout'])
    else:
        return


def fetch_candles0(instrument):
    params = {
        "count": 31,
        "granularity": "M1"
    }
    r = instruments.InstrumentsCandles(instrument = instrument, params = params)
    return client.request(r), instrument


def fetch_candles1(instrument):
    params = {
        "count": 1,
        "granularity": "M1"
    }
    r = instruments.InstrumentsCandles(instrument = instrument, params = params)
    return client.request(r), instrument


def remember_candles(response, instrument):
    global glob_dict
    mas0, mas1, mas2, mas3, mas4 = list(glob_dict[tuple(instrument.split("_"))].values())
    candles = response.get('candles')
    
    for candle in candles:
        time = candle['time']
        time = time.split('.')[0].replace("T", " ")
        mas0 = mas0[1:] + [time]
        mas1 = mas1[1:] + [float(candle['mid']['h'])]
        mas2 = mas2[1:] + [float(candle['mid']['l'])]
        mas3 = mas3[1:] + [float(candle['mid']['c'])]
        mas4 = mas4[1:] + [float(candle['volume'])]
    
    dc = {"dt": mas0, "high": mas1, "low": mas2, "close": mas3, "volume": mas4}
    """
    file_name = f'bbb/{instrument.split("_")[0] + instrument.split("_")[1]}_bbb.txt'
    file_read = open(file_name, "w+", encoding = "utf-8")
    # file_read.write("\n".join(mas0))
    file_read.write(str(pd.DataFrame(dc)))
    file_read.close()
    """
    glob_dict[tuple(instrument.split("_"))] = dc


def remember_candles_hard(response, instrument):
    global glob_dict
    mas0, mas1, mas2, mas3, mas4 = [], [], [], [], []
    candles = response.get('candles')


    for candle in candles:
        time = candle['time']
        time = time.split('.')[0].replace("T", " ")
        mas0.append(time)
        mas1.append(float(candle['mid']['h']))
        mas2.append(float(candle['mid']['l']))
        mas3.append(float(candle['mid']['c']))
        mas4.append(float(candle['volume']))

    
    dc = {"dt": mas0, "high": mas1, "low": mas2, "close": mas3, "volume": mas4}
    """
    file_name = f'bbb/{instrument.split("_")[0] + instrument.split("_")[1]}_bbb.txt'
    file_read = open(file_name, "w+", encoding = "utf-8")
    # file_read.write("\n".join(mas0))
    file_read.write(str(pd.DataFrame(dc)))
    file_read.close()
    """
    glob_dict[tuple(instrument.split("_"))] = dc


RATE_LIMIT = 60
TIME_PERIOD = 60

@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)

def parse_oanda(message):
    global chat_states, termintate, registered_users, necessary, starting_point, predicting, accounted, currency_pairs, glob_dict, dict_time_frame

    chat_id = message.chat.id
    glob_dict = {x: {"dt": [], "high": [], "low": [], "close": [], "volume": []} for x in registered_users[chat_id]["curr"]}

    #print("parse_oanda", accounted, predicting)

    current_time = Ddtime.now()
    #print(current_time)
    time.sleep(60 - current_time.second + global_waiting)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_candles0, pair[0] + "_" + pair[1]) for pair in registered_users[chat_id]["curr"]]
        for future in as_completed(futures):
            response, instrument = future.result()
            remember_candles_hard(response, instrument)

    current_time = list(map(int, Ddtime.now().strftime("%M:%S").split(":")))
    print(current_time)
    
    #print(glob_dict)
    accounted = 1
    predicting = 0
    schet = 1

    while chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users:
        if predicting == 0:
            continue
        current_time = Ddtime.now()
        time.sleep(60 - current_time.second + global_waiting)
        
        current_time = list(map(int, Ddtime.now().strftime("%M:%S").split(":")))
        print(current_time)
        try:
            if schet % 31 == 0:
                schet = 1
                with ThreadPoolExecutor() as executor:
                    futures = [executor.submit(fetch_candles0, pair[0] + "_" + pair[1]) for pair in registered_users[chat_id]["curr"]]
                    for future in as_completed(futures):
                        response, instrument = future.result()
                        remember_candles_hard(response, instrument)
            else:
                
                schet += 1
                with ThreadPoolExecutor() as executor:
                    futures = [executor.submit(fetch_candles1, pair[0] + "_" + pair[1]) for pair in registered_users[chat_id]["curr"]]
                    for future in as_completed(futures):
                        response, instrument = future.result()
                        remember_candles(response, instrument)
            predicting = 0
            accounted = 1
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")



def calculate_indicator(indicator, period, df):
    global abstract
    method_function = getattr(abstract, indicator)
    window = period + 1
    values = []

    for w in df.rolling(window=window):
        if len(w) == window:
            val = method_function(w, timeperiod=period)
            values.append(val.iloc[-1])
        else:
            values.append(np.nan)

    new_df = pd.DataFrame(values, columns=[indicator+'_'+str(period)])
    new_df.set_index(df.index, inplace=True)

    return new_df



def set_features(data, ifAbstract):
    indicators = []

    CCI_period = 14
    RSI_period = 14
    MFI_period = 30
    DM_period = 12
    slow_EMA_period = 26
    fast_EMA_period = 12
    WILLR_period = 14
    MAX_MIN_period = 14
    SMA_period = 18
    ATR_period = 14
    Chaikin_period = 2 #в AD период не важен


    indicators.append(calculate_indicator(indicator='CCI', period=CCI_period, df=data))
    indicators.append(calculate_indicator(indicator='DX', period=DM_period, df=data))
    indicators.append(calculate_indicator(indicator='MFI', period=MFI_period, df=data))
    indicators.append(calculate_indicator(indicator='MINUS_DM', period=DM_period, df=data))
    indicators.append(calculate_indicator(indicator='PLUS_DM', period=DM_period, df=data))
    indicators.append(calculate_indicator(indicator='RSI', period=RSI_period, df=data))
    indicators.append(calculate_indicator(indicator='WILLR', period=WILLR_period, df=data))
    indicators.append(calculate_indicator(indicator='MAX', period=RSI_period, df=data))
    indicators.append(calculate_indicator(indicator='MIN', period=WILLR_period, df=data))
    indicators.append(calculate_indicator(indicator='EMA', period=slow_EMA_period, df=data))
    indicators.append(calculate_indicator(indicator='EMA', period=fast_EMA_period, df=data))


    df_features = pd.concat(indicators, axis=1)


    df_features['MAX/MIN'] = df_features['MAX_'+str(MAX_MIN_period)] / df_features['MIN_'+str(MAX_MIN_period)]
    df_features['MACD/close'] = (df_features['EMA_'+str(slow_EMA_period)] - df_features['EMA_'+str(fast_EMA_period)]) / data['close']

    df_features.drop(columns=['MAX_'+str(MAX_MIN_period), 'MIN_'+str(MAX_MIN_period), 'EMA_'+str(slow_EMA_period), 'EMA_'+str(fast_EMA_period)], axis=1, inplace=True)
    
    data = pd.concat([data, df_features], axis=1)

    # COMPLEX FEATURES

    # 1) Первые разности
    for col in data.columns:
        if col.endswith('close') or col.endswith('volume'):  # Рассмотрим только столбец close и volume
            data[f'delta_p1_{col}'] = data[col].diff(1)  # Первые разности, periods = 1
            data[f'delta2_p1_{col}'] = data[f'delta_p1_{col}'].diff(1)  # Вторые разности, periods = 1
            data[f'delta_p2_{col}'] = data[col].diff(2)  # Первые разности, periods = 2
            data[f'delta2_p2_{col}'] = data[f'delta_p1_{col}'].diff(2)  # Вторые разности, periods = 2
    
    # 2) продвинутые разности
    data['avg_сlose_3 - avg_close_3_shift_3'] = data['close'].rolling(window=3).mean() - data['close'].shift(3).rolling(window=3).mean()
    data['avg_сlose_5 - avg_close_5_shift_3'] = data['close'].rolling(window=5).mean() - data['close'].shift(3).rolling(window=5).mean()
    data['avg_сlose_5 - avg_close_5_shift_5'] = data['close'].rolling(window=5).mean() - data['close'].shift(3).rolling(window=5).mean()
    
    # 3) Произведения и отношения фичей
    features_to_multiply_and_divide = [
        'delta_p1_close', 'delta2_p1_close', 'delta_p1_volume', 'delta2_p1_volume',
        'delta_p2_close', 'delta2_p2_close', 'delta_p2_volume', 'delta2_p2_volume',
        
        'CCI_'+str(CCI_period), 'DX_'+str(DM_period), 'MFI_'+str(MFI_period),
        'MINUS_DM_'+str(DM_period), 'PLUS_DM_'+str(DM_period), 'RSI_'+str(RSI_period),
        'WILLR_'+str(WILLR_period), 'MAX/MIN',
        #'delta_p1_CCI', 'delta_p1_DX', 'delta_p1_MFI', 'delta_p1_MINUS_DM', 'delta_p1_PLUS_DM', 'delta_p1_RSI', 'delta_p1_WILLR', 'delta_p1_MAX/MIN',
        #'delta_p2_CCI', 'delta_p2_DX', 'delta_p2_MFI', 'delta_p2_MINUS_DM', 'delta_p2_PLUS_DM', 'delta_p2_RSI', 'delta_p2_WILLR', 'delta_p2_MAX/MIN',
    ] # Список фичей для произведения и деления

    
    for i, feature1 in enumerate(features_to_multiply_and_divide):
        for j, feature2 in enumerate(features_to_multiply_and_divide):
            if i < j:
                data[f'{feature1}_x_{feature2}'] = data[feature1] * data[feature2]

    # 4) Логарифмирование
    for col in data.columns:
        if col.endswith('close') or col.endswith('volume'):  # Рассмотрим только цены и объемы
            data[f'log_{col}'] = data[col].apply(lambda x: np.log(x) if x > 0 else 0)  # Логарифмирование
    
    # 5) сумма и разность фичей 
    features_to_sum_and_minus = [
        'CCI_'+str(CCI_period), 'DX_'+str(DM_period), 'MFI_'+str(MFI_period),
        'MINUS_DM_'+str(DM_period), 'PLUS_DM_'+str(DM_period), 'RSI_'+str(RSI_period),
        'WILLR_'+str(WILLR_period), 'MAX/MIN',   
    ]
    
    for i, feature1 in enumerate(features_to_sum_and_minus):
        for j, feature2 in enumerate(features_to_sum_and_minus):
            if i < j:  # Избежать дублирования
                data[f'{feature1}_+_{feature2}'] = data[feature1] + data[feature2]  # сумма фичей
                data[f'{feature1}_-_{feature2}'] = data[feature1] - data[feature2]  # разность фичей
                data[f'{feature2}_-_{feature1}'] = data[feature2] - data[feature1]  # разность фичей (другой порядок)
               

    # 6) Бинарные регрессоры
    # close
    data['(close_t-1 + close_t-2) > (close_t-3 + close_t-4)'] = (data['close'].shift(periods=1) + data['close'].shift(periods=2)) > (data['close'].shift(periods=3) + data['close'].shift(periods=4))
    data['(close_t + close_t-1) > (close_t-2 + close_t-3)'] = (data['close'] + data['close'].shift(periods=1)) > (data['close'].shift(periods=2) + data['close'].shift(periods=3))
    data['(close_t + close_t-1 + close_t-2) > (close_t-3 + close_t-4+ close_t-5)'] = (data['close'] + data['close'].shift(periods=1) + data['close'].shift(periods=2)) > (data['close'].shift(periods=3) + 
                                                                                                                                                                        data['close'].shift(periods=4) + 
                                                                                                                                                                          data['close'].shift(periods=5))

    
    
    # volume
    data['(volume_t-1 + volume_t-2) > (volume_t-3 + volume_t-4)'] = (data['volume'].shift(periods=1) + data['volume'].shift(periods=2)) > (data['volume'].shift(periods=3) + data['volume'].shift(periods=4))
    data['(volume_t + volume_t-1) > (volume_t-2 + volume_t-3)'] = (data['volume'] + data['volume'].shift(periods=1)) > (data['volume'].shift(periods=2) + data['volume'].shift(periods=3))
    data['(volume_t + volume_t-1 + volume_t-2) > (volume_t-3 + volume_t-4+ volume_t-5)'] = (data['volume'] + data['volume'].shift(periods=1) + data['volume'].shift(periods=2)) > (data['volume'].shift(periods=3) + 
                                                                                                                                                                                   data['volume'].shift(periods=4) + 
                                                                                                                                                                                   data['volume'].shift(periods=5))

    data['close_t/close_t-1'] = data['close'] / data['close'].shift(periods=1)
    data['close_t/close_t-2'] = data['close'] / data['close'].shift(periods=2)
    data['close_t/close_t-3'] = data['close'] / data['close'].shift(periods=3)

    data['log_(close_t/close_t-1)'] = data['close_t/close_t-1'].apply(lambda x: np.log(x) if x > 0 else 0)  
    data['log_(close_t/close_t-2)'] = data['close_t/close_t-2'].apply(lambda x: np.log(x) if x > 0 else 0)  
    data['log_(close_t/close_t-3)'] = data['close_t/close_t-3'].apply(lambda x: np.log(x) if x > 0 else 0)  

    return data.iloc[-1:]


def waiting_to_write(message, mas, para, tf):
    global chat_states, termintate, registered_users, glob_dict, dict_time_frame, storaging
    chat_id = message.chat.id
    if not(chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users):
        return
    date = Ddtime.now(timezone.utc)
    time.sleep((- timedelta(seconds= date.second) - timedelta(minutes= date.minute) + timedelta(minutes= time_delta*(date.minute//time_delta + 1))).total_seconds() +  + global_waiting + 4)

    if not(chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users):
        return

    if Ddtime.now(timezone.utc).strftime("%Y-%m-%d %H:%M") != mas[3][:mas[3].rfind(":")]:
        print("Oopsie:", Ddtime.now(timezone.utc).strftime("%Y-%m-%d %H:%M"), mas[3], sep = " ")

    mas[4] = glob_dict[para]["close"][-1]
    date = Ddtime.now(timezone.utc)
    time.sleep((- timedelta(seconds= date.second) + timedelta(minutes= time_delta)).total_seconds() + global_waiting + 4)

    if not(chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users):
        return

    if Ddtime.now(timezone.utc).strftime("%Y-%m-%d %H:%M") != mas[5][:mas[5].rfind(":")]:
        print("Oopsie:", Ddtime.now(timezone.utc).strftime("%Y-%m-%d %H:%M"), mas[5], sep = " ")

    mas[6] = glob_dict[para]["close"][-1]
    mas[7] = mas[6] - mas[4]
    mas.append(int(mas[7] >= 0))
    storaging.append(mas)
    

RATE_LIMIT = 60
TIME_PERIOD = 60

@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)
def storaging_oanda(message):
    global chat_states, termintate, registered_users, glob_dict, dict_all_curr, model, features, predicting, accounted, dict_time_frame, storaging
    chat_id = message.chat.id
    while chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users:
        if len(storaging) == 0:
            continue
        mas = storaging[0]
        try:
            #"""
            worksheet = spreadsheet.worksheet(mas[0])
            empty_row = len(worksheet.get_all_values())
            worksheet.insert_row(mas, empty_row + 1)
            worksheet = spreadsheet.worksheet("consolidate")
            empty_row = len(worksheet.get_all_values())
            worksheet.insert_row(mas, empty_row + 1)
            #"""
            storaging = storaging[1:]
        except gspread.exceptions.APIError as e:
            print(f"EXCEL request error: {e}")
            


def osnova_oanda(message):
    global chat_states, termintate, registered_users, glob_dict, dict_all_curr, model, features, predicting, accounted, dict_time_frame, storaging
    chat_id = message.chat.id
    actual_currencies = registered_users[chat_id]["curr"]
    lasty = actual_currencies[-1]
    while chat_states.get(chat_id, False) and not termintate.get(chat_id, False) and chat_id in registered_users:
        if accounted == 0:
            continue
        for para in actual_currencies:
            dfing = pd.DataFrame(glob_dict[para])
            dfing.drop(["dt"], axis= 1, inplace= True)
            dating_add = set_features(dfing, ifAbstract=True)
            sc = scaler[para]
            dating_add = pd.DataFrame(sc.transform(dating_add[features]), columns=dating_add[features].columns, index=dating_add[features].index)
            probab = model.predict_proba(dating_add[features])[:, 1]

            """
            file_name = f'probab/{para[0] + para[1]}_probab.txt'
            file_read = open(file_name, "r+", encoding = "utf-8")
            info = file_read.readlines()
            info = [x.strip() for x in info if x.strip() != ""] + [str(probab)]
            file_read.close()
            file_read = open(file_name, "w+", encoding = "utf-8")
            file_read.write("\n".join(info))
            file_read.close()
            """

            date = Ddtime.now(timezone.utc)
            if probab[0] >= 0.6:
                tf = time_delta*(date.minute//time_delta + 1)
                tf = date - timedelta(seconds= date.second) - timedelta(minutes= date.minute) + timedelta(minutes= tf)
                if dict_time_frame[para]["tf_buy"] != tf.strftime("%Y-%m-%d %H:%M:%S"):
                    dict_time_frame[para]["tf_pred"] = glob_dict[para]["dt"][-1]
                    dict_time_frame[para]["tf_buy"] = tf.strftime("%Y-%m-%d %H:%M:%S")
                    dict_time_frame[para]["tf_sell"] = (tf + timedelta(minutes= time_delta)).strftime("%Y-%m-%d %H:%M:%S")
                    dict_time_frame[para]["predicted"] = glob_dict[para]["close"][-1]
                    args = (message, list(dict_time_frame[para].values()), para, tf)
                    threading.Thread(target = waiting_to_write, args=args).start()
                    to_show = ["Время предсказания: " + str(glob_dict[para]["dt"][-1]), "Цена закрытия: " + str(glob_dict[para]["close"][-1])]
                    bot.send_message(chat_id, "Заходим:\n" + " -> ".join(para) + "\n" + "-"*10 + "\n" + "\n".join(to_show))    
        accounted = 0
        predicting = 1


### Часть с функцией остановы и запуском бота

In [ ]:
@bot.message_handler(commands=['stop'], func = lambda x: not termintate.get(x.chat.id, False))
def handle_stop(message):
    global chat_states, termintate, registered_users, necessary, has_started
    
    chat_id = message.chat.id
    
    if has_started == chat_id:
        has_started = ""
        
        for para in registered_users[chat_id]["curr"]:
            file_name = f"{chat_id}_{para[0] + para[1]}_ccc.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{para[0] + para[1]}_bbb.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{chat_id}_aaa.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
            file_name = f"{chat_id}_{para[0] + para[1]}_ddd.txt"
            if os.path.exists(file_name):
                os.remove(file_name)
                

        chat_states[chat_id] = False
        bot.reply_to(message, "Программа остановлена.\nДля начала работы введите /start,\nдля полного прекращения - /stopping.",
                    reply_markup=create_command_keyboard())
    elif has_started == "":
        bot.reply_to(message, "Для начала введите команду /start.")#, reply_markup=create_command_keyboard())
    else:
        bot.reply_to(message, "Команда воспроизводится на другом устройстве.\nПодождите, пока исполнение на нем закончится.")#, reply_markup=create_command_keyboard())


@bot.message_handler(func = lambda x: not termintate.get(x.chat.id, False))
def handle_messages(message):
    global chat_states, termintate, registered_users, necessary

    chat_id = message.chat.id

    if not chat_states.get(chat_id, False):
        bot.reply_to(message, "Введите /start",
                     reply_markup=create_command_keyboard())

In [ ]:
### Замер нагрузки на компьютер
tracemalloc.start()

bot.polling()

In [ ]:
print(tracemalloc.get_traced_memory())

tracemalloc.stop()